In [1]:
# imports 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 11:14:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/18 11:14:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Data Acquisition

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

In [5]:
# for demo purposes
from pydataset import data

case = spark.createDataFrame(data("cases"))

case.write.json("data/case_json", mode="overwrite")

# like much else in spark, there's multiple ways we could do this:
(
    case.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("data/case_csv")
)


Did you mean:
Pastes, cats, MCAS, cane, CNES, austres, cars, cake, datasets, cabbages, crabs


TypeError: 'NoneType' object is not iterable